In [1]:
# import pandas as pd
import markupsafe
#
markupsafe.soft_unicode = markupsafe.soft_str
import basedosdados as bd
#
from dotenv import get_key
#
#
BILLING_PROJECT_ID = get_key('.env', 'BILLING_PROJECT_ID')

In [ ]:
df_chamado_1746 = bd.read_query(
'''
SELECT
    id_chamado
    ,data_inicio
    ,id_bairro
    ,id_tipo
    ,tipo
    ,id_subtipo
    ,subtipo
FROM
    `datario.administracao_servicos_publicos.chamado_1746`
WHERE
  data_particao BETWEEN '2022-01-01' AND '2023-12-31'
''',
billing_project_id=BILLING_PROJECT_ID
)
df_chamado_1746.head()

In [9]:
df_bairro = bd.read_table(
    dataset_id='dados_mestres',
    table_id='bairro',
    billing_project_id=BILLING_PROJECT_ID,
    query_project_id='datario'
)
df_bairro.head()

Downloading: 100%|██████████| 164/164 [00:11<00:00, 14.58rows/s]


,id_bairro,nome,id_area_planejamento,id_regiao_planejamento,nome_regiao_planejamento,id_regiao_administrativa,nome_regiao_administrativa,subprefeitura,area,perimetro,geometry_wkt,geometry
0,2,Gamboa,1,1.1,Centro,1,Portuaria,Centro,1.112903e+06,4612.833630,POLYGON ((-43.18791509600138 -22.8931217212322...,"POLYGON((-43.1879150960014 -22.8931217212322, ..."
1,1,Saúde,1,1.1,Centro,1,Portuaria,Centro,3.638176e+05,2646.220568,POLYGON ((-43.181151633502964 -22.895430284304...,"POLYGON((-43.181151633503 -22.8954302843042, -..."
2,4,Caju,1,1.1,Centro,1,Portuaria,Centro,5.347481e+06,19800.522524,MULTIPOLYGON (((-43.22522241788469 -22.8746498...,MULTIPOLYGON(((-43.196711909178 -22.8872495021...
3,3,Santo Cristo,1,1.1,Centro,1,Portuaria,Centro,1.684721e+06,6743.227885,POLYGON ((-43.194498082949806 -22.903378003392...,"POLYGON((-43.1944980829498 -22.9033780033923, ..."
4,161,Lapa,1,1.1,Centro,2,Centro,Centro,2.983258e+05,3849.181818,POLYGON ((-43.18166120770202 -22.9120798224676...,"POLYGON((-43.181661207702 -22.9120798224677, -..."


In [10]:
df_eventos = bd.read_table(
    dataset_id='turismo_fluxo_visitantes',
    table_id='rede_hoteleira_ocupacao_eventos',
    billing_project_id=BILLING_PROJECT_ID,
    query_project_id='datario'
)
df_eventos.head()

Downloading: 100%|██████████| 4/4 [00:00<00:00,  6.28rows/s]


,ano,data_inicial,data_final,evento,taxa_ocupacao
0,30-31/12 e 01/01 (2022-2023),2022-12-30,2023-01-01,Reveillon,0.9251
1,18/02 a 21/02 de 2023,2023-02-18,2023-02-21,Carnaval,0.9554
2,08/09 a 11/09 de 2022,2022-09-08,2022-09-11,Rock in Rio,0.9451
3,02/09 a 04/09 de 2022,2022-09-02,2022-09-04,Rock in Rio,0.8184
